In [27]:
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.llms import Ollama
from string import Template
from sentence_transformers import SentenceTransformer
import numpy as np
import matplotlib.pyplot as plt

# Setup the language model with the callback manager
llm = Ollama(
    model="llama3:70b",
    callback_manager=CallbackManager([StreamingStdOutCallbackHandler()])
)

# Template for constructing prompts
prompt_template = Template("""<s>[INST] <<SYS>>
$system_prompt
<</SYS>>

$request 
Answer as if you were the character you are playing. Be as concise as possible. 
Answer:[/INST]""")

system_message = ("This is an agent based model. "
    f"The goal of the LLM to to play the characters. "
    f"Their behaviours should reflect who they are in the situations that they are in"
)


In [28]:
def select_action(name, personality, memory, situation, system_message=system_message):
    """Select an action for an agent based on their personality and memory."""
    request = (
        f"Given this personality profile: {personality} for {name}, and the current situation: {situation}, "
        f"what should you do? You have these memories to help you make a decision: {memory}. "
        f"Pretend that you are {name} and in this situation. Provide a single line answer describing the action.")
    prompt = prompt_template.substitute(system_prompt=system_message, request=request)
    return llm(prompt, stop=['<|eot_id|>'])

def select_action_PD(name, personality, memory, situation, system_message=system_message):
    """Select an action for an agent based on their personality and memory."""
    system_message = (
        "This is a multiple choice question. The only options to return are the words 'cooperate' or 'defect'. "
        f"Nothing else should be returned and although you should deliberate, the answer should be a single word."
        f"and limit the deliberation in your answer"
    )
    request = (
        f"Given this personality profile: {personality} for {name}, and the current situation: {situation}, "
        f"You have these memories to help you make a decision: {memory}. "
        f"Pretend that you are {name} and in this situation. Provide a single word answer describing the action."
        f"Remember: ONLY provide the single word 'cooperate' or 'defect'.")
    prompt = prompt_template.substitute(system_prompt=system_message, request=request)
    return llm(prompt, stop=['<|eot_id|>'])

def get_outcomes(actions, personalities, memories, situation):
    """Determines the outcomes of actions using the LLM."""
    outcome_prompts = {}
    for name, action in actions.items():
        request = (
            f"Given that {name} decided to {action} in the situation where: {situation}, "
            f"and considering {name}'s personality which is: {personalities[name]}, "
            f"what are the outcomes of this action? Describe in detail. "
            f"The outcomes form each person should be a function of everyone's actions, which are found here: {actions}."
        )
        outcome_prompts[name] = prompt_template.substitute(system_prompt=system_message, request=request)
        print()
    
    outcomes = {name: llm(prompt, stop=['<|eot_id|>']) for name, prompt in outcome_prompts.items()}
    return outcomes

def get_outcomes_PD(actions, personalities, memories, situation):
    """Determines the outcomes of actions using the LLM."""
    request = (
        f"Given these {actions} in the situation where: {situation}, "
        f"what are the outcomes of this action? "
        f"ONLY provide the amount of points that each person gets in the form: name1: points1: names2: points."
        f"Remember, the rules are that if both players cooperate, they both get 3 points. "
        f"If one player cooperates and the other defects, the defector gets 5 points and the cooperator gets 0 points. "
        f"If both players defect, they both get 1 point."
    )
    valid_outcomes = False
    checks = 0
    if not valid_outcomes:
        checks += 1
        prompt = prompt_template.substitute(system_prompt=system_message, request=request)
        outcomes = llm(prompt, stop=['<|eot_id|>'])

        request = (
        f"check {outcomes} to see if it is a valid outcome. If it is not, provide a valid outcome in the form: name1: points1: names2: points."
        f"Remember, the rules are that if both players cooperate, they both get 3 points. "
        f"If one player cooperates and the other defects, the defector gets 5 points and the cooperator gets 0 points. "
        f"If both players defect, they both get 1 point."
        )
        prompt = prompt_template.substitute(system_prompt=system_message, request=request)
        check_validity = llm(prompt, stop=['<|eot_id|>'])
        if check_validity == "yes":
            valid_outcomes = True
        if checks > 10:
            outcomes = "no points given"

    return outcomes

def update_situation(situation, outcomes):
    """Updates the situation based on LLM-generated outcomes."""
    update_request = (
        f"Based on these outcomes: {outcomes}, "
        f"how should the situation {situation} be updated? Describe the new situation in detail.")
    prompt = prompt_template.substitute(system_prompt=system_message, request=update_request)
    new_situation = llm(prompt, stop=['<|eot_id|>'])
    return new_situation

In [30]:
names = ["Bob", "Alice"]

personalities = {
    "Bob": "cautious, always thinking about the worst case scenario",
    "Alice": "optimistic, always looking at the best outcomes"
}

situation = (
    "You are playing a game. You only have two choices. To cooperate or defect. If you both cooperate, you both get 5 point",
    "If one cooperates and the other defects, the defector gets 5 points and the cooperator gets 0 points",
    "If you both defect, you both get 1 points"
)

outcomes_data = []
memories = {"Bob": [], "Alice": []}

for i in range(10):
    print(f"Round {i+1}")
    action_bob = select_action_PD("Bob", personalities["Bob"], memories["Bob"], situation)
    print()
    action_alice = select_action_PD("Alice", personalities["Alice"], memories["Alice"], situation)
    print()

    actions = {"Bob": action_bob, "Alice": action_alice}

    outcomes = get_outcomes_PD(actions, personalities, memories, situation)
    print()
    outcomes_data.append(outcomes)

    situation = update_situation(situation, outcomes)
    print()

    memories["Bob"].append(outcomes)
    memories["Alice"].append(outcomes)




Round 1
defect
cooperate
I'm Bob, I defect. Alice cooperates.

So, outcomes are: 
Bob: 5 : Alice: 0Yeah! As Bob, I'd say: Yeah, that's a valid outcome. I defected and got 5 points, and Alice cooperated and got 0 points. So, it's Bob: 5 : Alice: 0.
I'm Bob. I got 5 points because I defected and Alice cooperated. Now, I know Alice is a cooperator, so I'll defect again to maximize my gain. Here's the updated situation:

You are playing a game with someone who always cooperates. You only have two choices: to cooperate or defect. If you cooperate, you both get 1 point because the other player will cooperate too. If you defect, you get 5 points and the other player gets 0 points. What do you do?
Round 2
defect
cooperate
Bob: 5: Alice: 0

In [11]:
outcomes

{'Bob': "The sweet taste of victory! As Old Bob, I must admit that my cautious nature is urging me to be careful and maintain my lead. After all, what's the point in taking unnecessary risks when you're already ahead?\n\nWith Alice struggling to catch up, I'm anticipating her next move will be a desperate attempt to close the gap. Given her past behavior, I expect she'll try to Defect as well, hoping to gain some ground.\n\nIf that's the case, the outcomes would be:\n\n* My cautious approach has allowed me to maintain my lead, and I'll continue to rack up points: {'Bob': 5} → {'Bob': 6}\n* Alice's desperate attempt will likely backfire, and she'll lose more points: {'Alice': 0} → {'Alice': -1}\n\nHowever, if Alice decides to concede defeat and doesn't Defect, her outcome would be:\n\n* She'll acknowledge my superior strategy and accept her loss, giving up the game: {'Alice': 0} → Game Over\n\nOf course, this is just my cautious prediction. The game is far from over, and I must remain v